In [ ]:
!pip install openpyxl
!pip install pandas

In [3]:
import openpyxl
import pandas as pd
import glob

In [69]:
import_file_path = 'your_file_location'
excel_sheet_name = '全部員'
export_file_path = 'your_file_location'

In [70]:
df_order = pd.read_excel(import_file_path, sheet_name = excel_sheet_name)

In [ ]:
df_order

In [72]:
people_name = df_order['氏名'].unique()
people_location = df_order['住所'].unique()
people_birthday = df_order['生年月日'].unique()

In [ ]:
people_name
people_location
people_birthday

In [ ]:
for person_name in people_name:
    print(person_name)
for person_location in people_location:
    print(person_location)
for person_birthday in people_birthday:
    print(person_birthday)


In [ ]:
from datetime import datetime
for person_birthday in people_birthday:
        
    # 元の日付の文字列のリスト
    date_string = str(person_birthday)

    # 日付をパースしてdatetimeオブジェクトのリストを作成
    dates = [datetime.strptime(date_string, '%Y-%m-%d %H:%M:%S')]

    # 年、月、日をそれぞれ抽出
    years = [date.year for date in dates]
    months = [date.month for date in dates]
    days = [date.day for date in dates]

    # 結果の確認
    print("Years:", years)
    print("Months:", months)
    print("Days:", days)


In [ ]:
pip install python-docx

In [42]:
from docx import Document 

In [168]:
from docx.oxml import OxmlElement
from docx.shared import RGBColor
from docx.oxml.ns import qn

def set_cell_borders_black(cell):
    # セルの枠線の設定を行うためのプロパティを作成・設定
    tcPr = cell._element.get_or_add_tcPr()
    
    # 上の枠線
    top = OxmlElement('w:top')
    top.set(qn('w:val'), 'single')
    top.set(qn('w:sz'), '4')
    top.set(qn('w:color'), '000000')  # 黒色
    tcPr.append(top)

    # 左の枠線
    left = OxmlElement('w:left')
    left.set(qn('w:val'), 'single')
    left.set(qn('w:sz'), '4')
    left.set(qn('w:color'), '000000')  # 黒色
    tcPr.append(left)

    # 下の枠線
    bottom = OxmlElement('w:bottom')
    bottom.set(qn('w:val'), 'single')
    bottom.set(qn('w:sz'), '4')
    bottom.set(qn('w:color'), '000000')  # 黒色
    tcPr.append(bottom)

    # 右の枠線
    right = OxmlElement('w:right')
    right.set(qn('w:val'), 'single')
    right.set(qn('w:sz'), '4')
    right.set(qn('w:color'), '000000')  # 黒色
    tcPr.append(right)


In [173]:

from docx.shared import Pt,Inches
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.oxml import OxmlElement

for i in range(len(people_birthday)):
    
    # 新しいドキュメントを作成
    doc = Document()
    chiji_name = "name"

    # タイトルを追加し、中央揃えに設定
    doc.add_paragraph('第38号の5様式\n\n')
    doc.add_paragraph('comment\n').alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    doc.add_paragraph('　　石川県知事　'+chiji_name+'様\n')
    # 本文を追加
    doc.add_paragraph('coment')

    table = doc.add_table(rows=9, cols=2)  # 行数を9に修正（備考含む）

    for row in table.rows:
        for cell in row.cells:
            set_cell_borders_black(cell)
    # 列幅を設定
    for row in table.rows:
        row.cells[0].width = Inches(1.4)
        row.cells[1].width = Inches(4.3)
    # 元の日付の文字列のリスト
    date_string = str(people_birthday[i])
    # 日付をパースしてdatetimeオブジェクトのリストを作成
    date = datetime.strptime(date_string, '%Y-%m-%d %H:%M:%S')

    # 年、月、日をそれぞれ抽出
    years = date.year
    months = date.month
    days = date.day

    # セルのテキストを設定
    rows_contents = [
        ('利用ゴルフ場名', ''),
        ('利用年月日', '年　　　月　　　日'),
        ('住所', people_location[i]),
        ('区分', '□　メンバー　　□　ビジター'),
        ('氏名', people_name[i]),
        ('生年月日', '大・昭・平'+str(years-1988)+'年'+str(months)+'月'+str(days)+'日生'),
        ('非課税等適用区分', '□70歳以上　□18歳未満　□障害者等\n□教育活動等　□国民スポーツ大会　□スポーツマスターズ等'),
        ('証明書の種類', '□運転免許証　□学生証　□職員証　□パスポート\n□障害者手帳等　□学校長の証明　□教育委員会の証明\n□その他(　　　　　　　　)'),
        ('備考','')
    ]

    for row, (label, content) in zip(table.rows, rows_contents):
        row.cells[0].text = label
        row.cells[1].text = content
        for cell in row.cells:
            paragraphs = cell.paragraphs
            for paragraph in paragraphs:
                paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
                for run in paragraph.runs:
                    run.font.size = Pt(10)


    remarks = '''備考　1　該当する□の中にレ点を付けてください。
　　　2　70歳以上、18歳未満及び障害者等の方は、この申請書を、利用するゴルフ場が最初の\n　　　利用である場合にゴルフ場に提出してください。また、受付の際に非課税利用に該当する\n　　　ことを証明する証明書をゴルフ場に提示してください。
　　　3　教育活動等、国民スポーツ大会、スポーツマスターズ等の利用の場合は、利用の都度\n　　　この申請書を提出してください。その際には、受付に非課税・課税免除利用に該当するこ\n　　　とを証明する証明書をゴルフ場に提出してください。
　　　4　この申請書を提出しない場合、2又は3の証明書を提示又は提出しない場合は、非課\n　　　税・課税免除の適用を受けられない場合があります。

'''
    doc.add_paragraph(remarks).aligment = WD_PARAGRAPH_ALIGNMENT.JUSTIFY
    


    # ファイルを保存

    file_path = 'your_file_location'
    doc.save(file_path)

    file_path


In [174]:
from docx  import Document 
from docx.shared import Pt

for i in range(len(people_birthday)):
    doc = Document('your_file_location')
    
    for paragraph in doc.paragraphs:
        for run in paragraph.runs:
            run.font.size = Pt(10)
            run.font.name = 'ＭＳ 明朝' 
    
    doc.save('your_file_location')        